In [433]:
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.regression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import openpyxl

In [799]:
data = pd.read_excel('BASEi.xlsx', index_col=0)
df_dummies = pd.get_dummies(data['Mes'])
df_dummies.columns = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
data = data.join(df_dummies)
#Split data into train and test the X is all the columns minus 'Energia GBA' which is the target
X = data.drop(['Energia GBA', 'Energia PBA','Clima GBA', 'lag GBA', 'Año', 'Mes'], axis=1)
y = data['Energia PBA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X.head()

,Hora,Clima PBA,lag PBA,Laboral,Semi Laboral,Feriado o domingo,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,0,14.0,1544.9,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
1,1,13.2,1425.9,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,2,12.1,1387.8,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,3,11.2,1327.7,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
4,4,10.4,1268.6,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0


In [802]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=3600,
    tmp_folder='autosklearn_regression_tmp')
#exclude =  {'regressor': ["gradient_boosting"]}
#, include = {'regressor': ["extra_trees", "k_nearest_neighbors"]}, exclude=None
automl.fit(X_train, y_train, dataset_name='Energia')

AutoSklearnRegressor(per_run_time_limit=360,
                     tmp_folder='autosklearn_regression_tmp')

In [803]:
print(automl.leaderboard())

          rank  ensemble_weight               type      cost   duration
model_id                                                               
152          1             0.46                mlp  0.178332  46.764348
23           2             0.28  gradient_boosting  0.183894   4.161529
159          3             0.04                mlp  0.184897  17.156340
104          4             0.12                sgd  0.189496   2.812820
162          5             0.08                mlp  0.190712  32.512675
153          6             0.02                mlp  0.221111  12.692651


In [804]:
#Predict the test set
predictions = automl.predict(X_test)
#Print the mean absolute error
print("Mean absolute error: %.2f" % sklearn.metrics.mean_absolute_error(y_test, predictions))

Mean absolute error: 71.14


In [805]:
#Import xlsx
data_oct = pd.read_excel('oct.xlsx', index_col=0)
#Split data into train and test the X is all the columns minus 'Energia GBA' which is the target
X_oct = data_oct.drop(['Energia GBA', 'Energia PBA', 'Clima GBA', 'lag GBA', 'Tipo Dia', 'Año', 'Mes'], axis=1)
y_oct = data_oct['Energia PBA']

In [806]:
#Predict the test set
predictions = automl.predict(X_oct)
#Print the mean absolute error

print("RMSE: %.2f" % sklearn.metrics.mean_squared_error(y_oct, predictions, squared = False))
print("MSE: %.2f" % sklearn.metrics.mean_squared_error(y_oct, predictions))
print("MAE: %.2f" % sklearn.metrics.mean_absolute_error(y_oct, predictions))
predictions = pd.DataFrame(predictions)
#Export predictions
predictions.to_csv('predictions.csv')

RMSE: 145.85
MSE: 21272.89
MAE: 127.21
